# ANNUITY CALCULATOR

In this notebook I use the simple mortgage calculator to generate mortgage cashflows.

In [1]:
from financepy.products.bonds import *
from financepy.finutils import *
from financepy.products.rates import *

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 20:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## CREATE AN ANNUITY STREAM

Calculate the cashflows on an annuity

In [2]:
valuationDate = FinDate(20, 6, 2018)
settlementDate = valuationDate

In [3]:
maturityDate = settlementDate.addTenor("5Y")  # FinDate(20, 9, 2028)
coupon = 0.05
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
calendarType = FinCalendarTypes.WEEKEND
busDayAdjustType = FinBusDayAdjustTypes.FOLLOWING
dateGenRuleType = FinDateGenRuleTypes.BACKWARD
basisType = FinDayCountTypes.ACT_360
face = ONE_MILLION

In [4]:
annuity = FinBondAnnuity(maturityDate,coupon,frequencyType,calendarType,
                         busDayAdjustType,dateGenRuleType,basisType,face)

In [5]:
print(annuity)

OBJECT TYPE: FinBondAnnuity
MATURITY DATE: TUE 20 JUN 2023
FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
CALENDAR: FinCalendarTypes.WEEKEND
BUS_DAY_RULE: FinBusDayAdjustTypes.FOLLOWING
DATE_GEN_RULE: FinDateGenRuleTypes.BACKWARD



In [6]:
annuity.printFlows(settlementDate)

THU 20 DEC 2018 , 25416.666666666668
THU 20 JUN 2019 , 25277.777777777777
FRI 20 DEC 2019 , 25416.666666666668
SAT 20 JUN 2020 , 25416.666666666668
SUN 20 DEC 2020 , 25416.666666666668
SUN 20 JUN 2021 , 25277.777777777777
MON 20 DEC 2021 , 25416.666666666668
MON 20 JUN 2022 , 25277.777777777777
TUE 20 DEC 2022 , 25416.666666666668
TUE 20 JUN 2023 , 25277.777777777777


## We can value the flows by discounting them

First we build a Libor swap curve

In [7]:
depos = []
dcType = FinDayCountTypes.ACT_360
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinSwapTypes.PAY
swap1 = FinIborSwap(settlementDate,"1Y",swapType,0.0500,fixedFreq,dcType)
swap2 = FinIborSwap(settlementDate,"2Y",swapType,0.0500,fixedFreq,dcType)
swap3 = FinIborSwap(settlementDate,"3Y",swapType,0.0500,fixedFreq,dcType)
swap4 = FinIborSwap(settlementDate,"4Y",swapType,0.0500,fixedFreq,dcType)
swap5 = FinIborSwap(settlementDate,"5Y",swapType,0.0500,fixedFreq,dcType)
swaps = [swap1, swap2, swap3, swap4, swap5]

liborCurve = FinIborSingleCurve(valuationDate, depos, [], swaps)

In [8]:
swap5.value(settlementDate, liborCurve)

-4.3655745685100555e-10

In [9]:
swap5.printFixedLegPV()

START DATE: WED 20 JUN 2018
MATURITY DATE: TUE 20 JUN 2023
COUPON (%): 5.0
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE WED 20 JUN 2018
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
WED 20 JUN 2018          -            -   1.00000000            -            -
THU 20 DEC 2018  0.5083333     25416.67   0.97521333     24786.67     24786.67
THU 20 JUN 2019  0.5055556     25277.78   0.95116987     24043.46     48830.13
FRI 20 DEC 2019  0.5083333     25416.67   0.92759353     23576.34     72406.47
MON 22 JUN 2020  0.5138889     25694.44   0.90435659     23236.94     95643.41
MON 21 DEC 2020  0.5055556     25277.78   0.88206007     22296.52    117939.93
MON 21 JUN 2021  0.5055556     25277.78   0.86031327     21746.81    139686.73
MON 20 DEC 2021  0.5055556     25277.78   0.83910262     21210.65    160897.38
MON 20 JUN 2022  0.5055556     25277.78   0.81841491     20687.71    181585.09
TUE 20 

Here are the flows on the fixed leg of the 5Y swap

In [ ]:
swap5.printFloatLegPV()

In [ ]:
cleanValue = annuity.cleanPriceFromDiscountCurve(settlementDate, liborCurve)

In [ ]:
dirtyValue = annuity.fullPriceFromDiscountCurve(settlementDate, liborCurve)

The price of the annuity is quoted based on a par notional

In [ ]:
print("CLEAN PRICE:", cleanValue)
print("DIRTY PRICE:", dirtyValue)
print("ACCRD VALUE:", annuity._accruedInterest)

Copyright (c) 2020 Dominic O'Kane